In [1]:
from web3 import Web3, HTTPProvider
web3 = Web3(HTTPProvider("HTTP://192.168.200.215:7545"))

In [2]:
list = web3.eth.accounts

In [10]:
from solcx import compile_standard

In [11]:
# read sol file. 
with open("./contracts/receiveEther.sol") as f:
    data = f.read()
data    

'pragma solidity ^0.5.0;\n\ncontract receiveEther {\n    uint id = 0 ;\n\n    function () external payable  {\n        id = msg.value;\n    }\n\n    function getnumber() public view returns (uint){\n        return id;\n    }\n}'

In [12]:
input = {
    "language": "Solidity",
    "sources": {
        "receiveEther.sol": {
            "content": data
        }
    },
    "settings":
    {
        "outputSelection": {
            "*": {
                "*": [
                    "metadata", "evm.bytecode", "evm.bytecode.sourceMap"
                ]
            }
        }
    }
}

In [13]:
compiledSol = compile_standard(input) 

In [14]:
bytecode = compiledSol['contracts']['receiveEther.sol']['receiveEther']['evm']['bytecode']['object']

In [15]:
import json
abi = json.loads(compiledSol['contracts']['receiveEther.sol']['receiveEther']['metadata'])['output']['abi']

In [16]:
CalcContract = web3.eth.contract(abi=abi, bytecode= bytecode)

In [17]:
tx_hash = CalcContract.constructor().transact({
    "from":list[0]
})

In [18]:
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash) 

In [19]:
# The contract contains payable fallback function.
# the result is payable address. 
conAddr = tx_receipt.contractAddress

### send ether to contract address. 

In [20]:
web3.eth.sendTransaction({
    'from' :list[0],
    'to' : conAddr,
    'value': Web3.toWei(0.15, 'ether')
})

HexBytes('0x6d0296a1f66def4f90af77ad344be9d20d41272f01f8764000f1f51de52f9098')

In [21]:
Instance = web3.eth.contract(
    address=conAddr,
    abi = abi
)

In [22]:
# interact with the SC 
Instance.functions.getnumber().call()

150000000000000000